In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [90]:
#Scraping the top5 leagues in Europe
leagues = ['EPL','La_Liga','Bundesliga','Serie_A','Ligue_1']
col_names=['player_id','goals']

#Lists for storing Player data
player_id=[]
goals=[]

#Looping over every league
for league in leagues:
    base_url = 'https://understat.com/league/'
    url = base_url+league
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
    
    #Player data is in scripts[3]
    strings = scripts[3].string
    ind_start = strings.index("('")+2 
    ind_end = strings.index("')") 
    
    #Get JSON data
    json_data = strings[ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

    #convert string to json format
    data = json.loads(json_data)

    #Get goals and id from json data
    for index in range(len(data)):
        for key in data[index]:
            if key=='id':
                player_id.append(data[index][key])
            if key == 'goals':
                goals.append(data[index][key])


df = pd.DataFrame(list(zip(player_id,goals)))
df.columns = col_names

df['goals'] = df['goals'].astype(int)
#Only considering the players who scored more than 5 goals
df = df[df['goals']>=5]
print(df)


https://understat.com/league/EPL
https://understat.com/league/La_Liga
https://understat.com/league/Bundesliga
https://understat.com/league/Serie_A
https://understat.com/league/Ligue_1
2759 2759
     player_id  goals
0          647     23
1         1250     22
2         1228     18
3          453     17
4          822     17
5         5555     16
6          755     15
7         8865     14
8          314     13
9         3277     13
10         468     12
11         620     12
12         986     12
13        4456     12
14         522     11
15         556     11
16         838     11
17        2251     11
18        7153     11
19         318     10
20         531     10
21         606     10
22         618     10
23        3294     10
24        5612     10
25        5656     10
26        8288     10
27         482      9
28         558      9
29         750      9
...        ...    ...
2222      5733      7
2223      6019      7
2224      6238      7
2225      6846      7
2226      7278

In [91]:
#Get updated Player ids
player_ids = df['player_id'].tolist()
player=[]
X=[]
Y=[]
xG=[]
situation=[]
season=[]
shotType=[]
h_team=[]
a_team=[]
player_assisted=[]
result=[]

#Loop over every player
for player_id in player_ids:
    #Player data scraped from
    base_url = 'https://understat.com/player/'
    url = base_url+player_id
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
    #Player shot data is stored in scripts[3]
    strings = scripts[3].string
    ind_start = strings.index("('")+2 
    ind_end = strings.index("')") 
    json_data = strings[ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

    #convert string to json format
    data = json.loads(json_data)

    
    for index in range(len(data)):
        for key in data[index]:
            if key=='player':
                player.append(data[index][key])
            if key == 'X':
                X.append(data[index][key])
            if key == 'Y':
                Y.append(data[index][key])
            if key == 'xG':
                xG.append(data[index][key])
            if key == 'season':
                season.append(data[index][key])
            if key == 'shotType':
                shotType.append(data[index][key])
            if key == 'situation':
                situation.append(data[index][key])
            if key == 'h_team':
                h_team.append(data[index][key])
            if key == 'a_team':
                a_team.append(data[index][key])
            if key == 'player_assisted':
                player_assisted.append(data[index][key])
            if key == 'result':
                result.append(data[index][key])
                
#Storing the shots data in new dataframe          
col_names=['player','X','Y','xG','situation','season','shotType','h_team','a_team','player_assisted','result']
df1 = pd.DataFrame(list(zip(player,X,Y,xG,situation,season,shotType,h_team,a_team,player_assisted,result)))
df1.columns = col_names
print(df1)

            player                     X                    Y  \
0       Harry Kane                 0.795  0.32099998474121094   
1       Harry Kane  0.029000000953674318   0.5820000076293945   
2       Harry Kane    0.9059999847412109  0.46900001525878904   
3       Harry Kane    0.9169999694824219  0.46299999237060546   
4       Harry Kane    0.7609999847412109                0.415   
5       Harry Kane     0.980999984741211   0.5579999923706055   
6       Harry Kane                 0.915   0.5740000152587891   
7       Harry Kane                  0.92   0.5359999847412109   
8       Harry Kane    0.8080000305175781   0.6020000076293945   
9       Harry Kane                 0.775   0.7930000305175782   
10      Harry Kane     0.715999984741211  0.45099998474121095   
11      Harry Kane    0.8219999694824218   0.5659999847412109   
12      Harry Kane     0.909000015258789   0.5309999847412109   
13      Harry Kane    0.9569999694824218   0.5279999923706055   
14      Harry Kane       

In [94]:
shots = df1
#Getting shots from the season 2020-21
shots = shots[shots['season']=='2020']
shots.to_csv('shots_europe_2020_2021.csv')